# Astronomical Telescopes and Instruments 2020: Exercise Set 8

Welcome to the eighth exercise class for ATI 2020. You have to hand in the completed notebook to Brightspace before the start of the next lecture. Information/things to think about will be in regular font, while exercises will be *italic*.

### Make sure your submitted notebook works when run from scratch! You can do this using the menu above 'Kernel/Restart & Run all'. There should be no errors!

*Upload the finished notebook to Brightspace before the start of the next lecture!*

These are the packages you need for this exercise set. You are welcome to add extra, but this should not be necessary.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import hcipy as hp
from scipy.optimize import curve_fit
import warnings

from tqdm import tqdm

%matplotlib inline

The final package *tqdm* allows you to add progress bars to *for* and *while* loops. You can disable it if you want to/can't install it, but you will have to remove a couple tqdm statements in the code below. Installation should be pretty easy through pip (details on *tqdm* are on https://pypi.org/project/tqdm/). 

While the topic for this week is adaptive optics, we will also be looking into models for atmospheric turbulence in this set. This set will rely heavily on *hcipy*, and you will have to do a bit more than changing a couple lines of code in a pre-written simulation. Make sure you understand the *hcipy* things explained in the 3rd set before starting this one. A first place to look when trying to do something in *hcipy* is in the documentation or the source code:  

Documentation: https://docs.hcipy.org/0.3.1/  
Source code: https://github.com/ehpor/hcipy/tree/master/hcipy

**Don't hesitate to send an email to the TAs asking for help if you get stuck on anything!**

The eventual goal of this set is to make a simple simulation of an ideal telescope surrounded by an AO-corrected atmosphere. To get there we will take a look at *hcipy*'s atmospheric model and use Zernike modes as a crude adaptive optics system.

#### Ideal wavefront

Let's start off easy with reusing some code from the 3rd set. If you didn't manage to get it working there, please let the TAs know so they can help you! We start with defining the parameters we're going to use in this set. All physical distances will implicitly have units of meters.

In [2]:
N = 256        # pixels along a side in the pupil grid
D = 4          # aperture diameter in the pupil grid
oversize = 1.1 # pupil grid diameter relative to the aperture diameter

q = 16         # number of pixels per lambda/d in the focal plane
num_airy = 24  # number of airy rings visible in the focal plane
F = 12         # F-number of the lens between the pupil and focal plane

wavelength = 550e-9 # wavelength of the light

*Create the following objects with the parameters defined above:*
* *Appropiately scaled pupil and focal grids.*
* *A circular aperture generator with diameter $D$.*
* *A Fraunhofer propagator with the correct focal length.*

In [ ]:
pupil_grid = 
focal_grid = 

aperture_generator = 
fraunhofer = 

*Make a plot of the pupil plane intensity and PSF of a flat (no phase error) wavefront at the entrance aperture.*  
Be careful with the units and scaling of the $x$ & $y$ axes. Unlike the 3rd set we do **not** have $f=\lambda=D$.

In [ ]:
# Your code goes here ...

We will be using some of the above variables for the rest of the set, so be careful you don't overwrite them with new objects/values.



#### Zernike modes

As mentioned in the lecture, Zernike modes are useful in describing low-order optical aberrations, and are therefore also commonly used in an adaptive optics setting. Unfortunately numbering Zernike modes can be a bit of a pain, so you will make a plot comparing the different Zernike modes and their numbering.

The infinite set of Zernike modes is a basis for all possible functions on the unit disk, similar to how every vector can be constructed from a set of basis vectors. Therefore Zernike modes form a *mode basis*. *hcipy* has a general framework for constructing and using a *mode basis* (though with a finite number of modes of course). 

We can make a Zernike mode basis with the following code:

In [ ]:
N_modes = 60

zernike_mode_generators = hp.make_zernike_basis(N_modes, D, None)
zernike_mode_evaluated = [mode(pupil_grid) for mode in zernike_mode_generators]
zernike_modes = hp.ModeBasis(zernike_mode_evaluated)

We can index the *ModeBasis* to get a single mode out:

In [ ]:
example_index = 7
example_mode = zernike_modes[example_index]

vmax = np.abs(example_mode).max()
vmin = -vmax

plt.figure()
hp.imshow_field(example_mode, grid_units = D, cmap = 'bwr', vmax = vmax, vmin = vmin)
plt.colorbar(label = "Mode value")
plt.xlabel(r"$x$ ($D$)")
plt.ylabel(r"$y$ ($D$)")
plt.title("Example Zernike mode")

plt.show()

If we take a slice rather than a single index, we get a new *ModeBasis* object with the selected set of modes:

In [ ]:
example_mode = zernike_modes[7]
example_reduced_basis = zernike_modes[7:10]

print("A single mode is of type {0}".format(type(example_mode)))
print("A set of modes is of type {0}".format(type(example_reduced_basis)))
print("The full ModeBasis is of type {0}".format(type(zernike_modes)))
print("The full ModeBasis has {0} modes, but the reduced set of modes has {1} modes".format(len(zernike_modes), len(example_reduced_basis)))

You may have noticed that only a single index is provided to the mode basis to get a Zernike mode. That is because this mode basis is ordered using the Noll ordering of Zernike modes. *hcipy* has functions to convert between the Noll index and the radial/azimuthal indices, which you can find at https://docs.hcipy.org/0.3.1/modules/mode_basis.html.

*Complete the code below to make a triangular plot of the first 28 Zernike modes. Each row must contain the modes with the same radial order. Include both the Noll index and radial/azimutal order for every mode.*

You can nicely distinguish between positive and negative values by using a 'diverging' colormap (see https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html). The 'bwr' colormap usually works pretty well. Look at the code above to see how you can get 0 (the background value) to be white.

In [ ]:
n_radial = 7
n_azimuthal_max = n_radial + 1

fig, axs = plt.subplots(n_radial, n_azimuthal_max, figsize = (n_azimuthal_max*2, n_radial*2))

mode_python_index = 0
for ii in range(n_radial):
    for jj in range(n_azimuthal_max):
        axs[ii,jj].axis('off')
        if jj <= ii: # ensure we are inside the triangle
            # Your code goes here ...
            # plot the relevant Zernike mode 
            # add the relevant indices to the subplot title
            
            mode_python_index += 1
          

A couple of the low order Zernike modes have a direct interpretation as a specific aberration. 

*To which optical aberrations do the Zernike modes with Noll indices 1, 2, 3, 4, 5, 6, and 11 correspond?*

**Your answer goes here**

#### Atmospheric phase screens

We will take a quick look at turbulent atmospheres, and there aren't any exercises in this section. Do read it carefully though, you need the information here for the next section! *hcipy* implements turbulent atmospheric layers through *AtmosphericLayer* and derived classes. A typical atmosphere model consists of multiple turbulent layers, each at a different height and with different noise scales. In this set we will keep things as simple as possible by using a single layer of infinitesimal thickness and infinite extent. This is implemented by the *InfiniteAtmosphericLayer* class. The code below creates an instance of this object.

In [ ]:
r0 = 0.2 # Fried parameter
L0 = 20  # upper structure scale
velocity = 10 #np.array([10,0]) # velocity vector of the layer

Cn2 = hp.Cn_squared_from_fried_parameter(r0, wavelength)
atmo_layer = hp.InfiniteAtmosphericLayer(pupil_grid, Cn2, L0, velocity)

The above code may generate a *VisibleDeprecationWarning* based on which version of *hcipy* and *numpy* you are using. This warning is harmless.

The $C_n^2$ is a parameter characterizing the turbulence integrated over the altitude of the atmosphere and is related to the Fried parameter. The Fried parameter is as defined in the lecture and gives the average coherence length of a patch of turbulence, $L_0$ gives an outer scaling length for the largest noise structures, and the velocity gives the motion the layer makes over time when simulating a finite exposure of a detector.

You can get the phase error applied by this layer using the '.phase_for(wavelength)' method. 

In [ ]:
atmo_phase = atmo_layer.phase_for(wavelength)

plt.figure()
hp.imshow_field(atmo_phase, grid_units = D)
plt.colorbar(label = "Phase error (rad)")
plt.xlabel(r"$x$ ($D$)")
plt.ylabel(r"$y$ ($D$)")

plt.show()

Atmospheric phase errors are modelled as random phase errors. The above phase screen is just one realization of this noise. It is clearly very different from for example white noise, which seems to have no structure at all. The apparent structure in the figure above comes from the fact that the noise behaves differently as a function of point separation. Two points which are very close don't have very different phases, but points which are far apart can have completely different phases.

#### Simple adaptive optics

With the Zernike polynomials and an atmospheric model we can create a simple simulation of a finite exposure of an AO-corrected atmosphere. Our adaptive optics model will be to just remove a set of Zernike modes for every instantaneous atmospheric phase screen. This ignores a myriad of effects discussed in the lectures (errors in wavefront estimation, AO time lag, and correction errors, for example), but this gives us a first order expectation of what an adaptive optics system does. First we have to be able to remove a set of Zernike modes from a Field. 

In [ ]:
def get_projection_matrix(mode_basis):
    '''
    Calculate the matrix which projects a Field onto components
    of the given ModeBasis. Calculating this matrix is expensive,
    but saves time later as coefficients don't have to be fitted
    with least-squares for every supplied field.
    
    Parameters
    ----------
    mode_basis : hcipy.ModeBasis
        The mode basis on which we want to project fields.
    
    Returns
    -------
    M : ndarray
        Projection matrix from an input Field onto a the
        given modes. Has shape (N, M), where N is the number
        of modes in the mode-basis, and M is the number of 
        points in the grid of the modes.
    '''
    return hp.inverse_tikhonov(mode_basis.transformation_matrix, 1E-7)

def remove_modes(input_field, modes, projection_matrix=None):
    '''
    Remove a set of modes from the input field.
    
    Parameters
    ----------
    input_field : hcipy.Field
        Input Field.
    modes : hcipy.ModeBasis
        Set of modes to be removed.
    projection_matrix : 2darray, optional
        Projection matrix from the input field to a set
        of mode coefficients. If not supplied, coefficients
        will be fitted with least-squares.
        
    Returns
    -------
    output_field : hcipy.Field
        Input field with the relevant modes removed.
    '''
    if projection_matrix is None:
        coeffs = modes.coefficients_for(input_field)
    else:
        coeffs = projection_matrix.dot(input_field)
        
    modes_field = modes.linear_combination(coeffs)
    
    output_field = input_field - modes_field
    
    return output_field

The largest errors in the wavefront are the large-scale errors. You may have noticed that the low order Zernike modes also have large-scale structure, with higher modes showing progressively finer details. This means that we can remove most of the wavefront variance just by correcting the first few Zernike modes. 

Let's check this for the 'atmo_phase' we calculated above. We set the phase outside the aperture to zero as we don't correct it there.

*Plot the variance of the atmospheric phase error as a function of the number of Zernike modes removed. Normalize the curve to the variance of the uncorrected atmospheric phase error.*

Hint: to project onto a smaller number of modes, get a *ModeBasis* with a reduced number of Zernike modes by creating a slice of the full *ModeBasis* and take the appropriate slice of the projection matrix calculated below.

In [ ]:
phase_error = atmo_phase * aperture_generator(pupil_grid)
full_projection_matrix = get_projection_matrix(zernike_modes)

# Your code goes here ...

Depending on your phase screen, you can see that removing the first few modes removes most of the variance, but removing more modes gives diminishing returns. 

Finally we get to the atmosphere simulation. We take a phase error from the atmospheric layer, remove the Zernike modes to model the adaptive optics system, and then propagate a wavefront with the remaining phase error to a focal plane, where we measure the resulting intensity pattern.

In [ ]:
def generate_aberrated_wavefront(phase_error, aperture_generator, wavelength):
    '''
    Generate a wavefront with a given phase aberration.
    
    Parameters
    ----------
    phase_error : hcipy.Field
        Phase aberration of the wavefront.
    aperture_generator : function
        Generator function for the aperture.
    wavelength : scalar
        Wavelength of the wavefront.
    
    Returns
    -------
    wavefront : hcipy.Wavefront
        The resulting wavefront.
    '''
    
    ideal_wavefront = aperture_generator(phase_error.grid)
    
    aberrated_wavefront = ideal_wavefront * np.exp(1j*phase_error)
    
    return hp.Wavefront(aberrated_wavefront, wavelength)

*Finish the below simulation to create two simulated detector images: one of the PSF with an un-corrected atmosphere, and one with the simple AO described above.*

Hint: The final detector images are sums of wavefronts at different simulation times, which are incoherent with each other. 

In [ ]:
T = 1 # seconds
N_t = 100
ts = np.linspace(0, T, N_t)

image_no_AO = 0
image_with_AO = 0
for ii, t in tqdm(list(enumerate(ts))):
    # simulate the time evolution of the atmospheric phase screen
    # this generates a warning which we ignore
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',
        message='The behavior of affine_transform with a 1-D array supplied for the matrix parameter has changed in SciPy 0.18.0.')
        
        atmo_layer.evolve_until(t) 
        atmo_screen = atmo_layer.phase_for(wavelength)
    
    # Your code goes here ...


# plot final focal plane images
plt.figure()
hp.imshow_field(np.log10(image_no_AO/image_no_AO.max()), grid_units = F*wavelength, vmin = -5)
plt.colorbar(label = r"$I/I_0")
plt.xlabel(r"$x$ ($f\lambda/D$)")
plt.ylabel(r"$y$ ($f\lambda/D$)")
plt.title("1 second exposure with uncorrected atmosphere")

plt.figure()
hp.imshow_field(np.log10(image_with_AO/image_with_AO.max()), grid_units = F*wavelength, vmin = -5)
plt.colorbar(label = r"$I/I_0")
plt.xlabel(r"$x$ ($f\lambda/D$)")
plt.ylabel(r"$y$ ($f\lambda/D$)")
plt.title("1 second exposure with simple AO-corrected atmosphere")
    

*Why does the AO only correct the center of the detector image? What would you need to do to correct a larger area?*

**Your answer goes here**

You created an image of a perfect PSF with the code at the start of the set. We can use this to calculate the Strehl ratio of the corrected image. Remember to scale the perfect PSF correctly so it has the same 'exposure time' as your PSF with AO. A Strehl bigger than 1 is impossible!

*Calculate the Strehl ratio of the PSF with AO-corrected atmosphere.*

In [ ]:
# Your code goes here ...

#### AO error budget

We will finish with a quick examination of some of the effects that the simulation you made above does not take into account. The final performance of an AO system depends on these sources of error. The William Herschel Telescope has a mirror diameter of 4.2 m, and typical values for atmospheric parameters are an $r_0$ of 0.25 m at 550 nm, a wind speed of 5 m/s, a time lag in the AO control of 2 ms, and other errors producing 40 nm rms wavefront error. Assume the deformable mirror has a continuous face sheet with a subaperture diameter of 0.4 m. 

One quick note: you can use a slightly different formula for the Strehl ratio than the one in the lectures. All the variances calculated there are already in units of rad$^2$, so you can calculate the Strehl using 
$$ S = e^{-\sigma_{\text{tot}}^2} $$
rather than the value given in the slides.

*What is the maximum average Strehl ratio that can be achieved by this AO system at 800 nm? What about at 1600 nm? What causes the difference between these two values?*

**Your answer goes here**

### Make sure your submitted notebook works when run from scratch! You can do this using the menu above 'Kernel/Restart & Run all'. There should be no errors!

*Upload the finished notebook to Brightspace before the start of the next lecture!*